In [28]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames",
    "uCSV",
    "OrderedCollections",
    "CSV",
    "XAM",
    "CodecZlib",
    "ProgressMeter",
    "StatsPlots",
    "Arrow"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

  Activating new project at `/local/scratch/cjprybol/slrmtmp.46910893/jl_YMJkXp`
   Resolving package versions...
    Updating `/local/scratch/cjprybol/slrmtmp.46910893/jl_YMJkXp/Project.toml`
  [295af30f] + Revise v3.6.4
    Updating `/local/scratch/cjprybol/slrmtmp.46910893/jl_YMJkXp/Manifest.toml`
  [da1fd8a2] + CodeTracking v1.3.6
  [aa1ae85d] + JuliaInterpreter v0.9.37
  [6f1432cf] + LoweredCodeUtils v3.0.5
  [bac558e1] + OrderedCollections v1.6.3
  [ae029012] + Requires v1.3.0
  [295af30f] + Revise v3.6.4
  [56f22d72] + Artifacts
  [2a0f44e3] + Base64
  [8ba89e20] + Distributed
  [7b1f6079] + FileWatching
  [b77e0a4c] + InteractiveUtils
  [76f85450] + LibGit2
  [8f399da3] + Libdl
  [d6f4376e] + Markdown
  [ca575930] + NetworkOptions v1.2.0
  [de0858da] + Printf
  [3fa0cd96] + REPL
  [9a3f8284] + Random
  [ea8e919c] + SHA v0.7.0
  [9e88b42a] + Serialization
  [6462fe0b] + Sockets
  [cf7118a7] + UUIDs
  [4ec0a83e] + Unicode
  [e37daf67] + LibGit2_jll v1.6.4+0
  [29816b5a] + LibSSH2

In [2]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data"

In [3]:
pilot_samples = readdir(joinpath(data_dir, "samples"), join=true)

14-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 20 bytes ⋯ "jects/me-cfs/data/samples/LO_FA"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 20 bytes ⋯ "jects/me-cfs/data/samples/LO_MO"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 22 bytes ⋯ "cts/me-cfs/data/samples/LO_PROB"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 22 bytes ⋯ "cts/me-cfs/data/samples/RA_PROB"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 22 bytes ⋯ "cts/me-cfs/data/samples/RI_1BRO"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 22 bytes ⋯ "cts/me-cfs/data/samples/RI_3BRO"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 20 bytes ⋯ "jects/me-cfs/data/samples/RI_FA"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 22 bytes ⋯ "cts/me-cfs/data/samples/RI_MAun"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 21 bytes ⋯ "ects/me-cfs/data/samples/RI_MGF"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 20 bytes ⋯ "jects/me-cfs/data/samples/RI_MO"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 22 bytes ⋯ "cts/me-cfs/data/samples/RI_MaUn"
 "/oak/stanford/scg/lab_mpsnyder/" ⋯ 22 bytes ⋯ "cts/me-cfs/data/sample

In [4]:
pilot_sample_table = DataFrames.DataFrame(
    sample_ID = String[],
    forward_reads = String[],
    reverse_reads = String[],
    filesize = Int[]
)

# run on just non-human reads first
for pilot_sample in pilot_samples
    qc_filtered_batches = filter(x -> occursin(r"trimgalore$", x), readdir(pilot_sample, join=true))
    for qc_filtered_batch in qc_filtered_batches
        forward_reads = filter(x -> occursin(r"\.unmapped\.1\.fq\.gz$", x), readdir(qc_filtered_batch, join=true))
        # forward_reads = filter(x -> occursin(r"1_val_1\.fq\.gz$", x), readdir(qc_filtered_batch, join=true))
        @assert length(forward_reads) == 1
        forward_reads = first(forward_reads)
        reverse_reads = filter(x -> occursin(r"\.unmapped\.2\.fq\.gz$", x), readdir(qc_filtered_batch, join=true))
        # reverse_reads = filter(x -> occursin(r"2_val_2\.fq\.gz$", x), readdir(qc_filtered_batch, join=true))
        @assert length(reverse_reads) == 1
        reverse_reads = first(reverse_reads)
        joint_filesize = filesize(forward_reads) + filesize(reverse_reads)
        row = (sample_ID = pilot_sample, forward_reads = forward_reads, reverse_reads = reverse_reads, filesize = joint_filesize)
        push!(pilot_sample_table, row)
    end
end
pilot_sample_table

Row,sample_ID,forward_reads,reverse_reads,filesize
,String,String,String,Int64
1,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H3TLGDSX7_L1_trimgalore/LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H3TLGDSX7_L1_trimgalore/LO_FA_CKDN230005699-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,9537348
2,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HNJDSX7_L2_trimgalore/LO_FA_CKDN230005699-1A_H5HNJDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HNJDSX7_L2_trimgalore/LO_FA_CKDN230005699-1A_H5HNJDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,2013165
3,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HVHDSX7_L1_trimgalore/LO_FA_CKDN230005699-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HVHDSX7_L1_trimgalore/LO_FA_CKDN230005699-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,1394841
4,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2_trimgalore/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2_trimgalore/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,396783
5,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO/LO_MO_CKDN230005698-1A_H3TLGDSX7_L1_trimgalore/LO_MO_CKDN230005698-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO/LO_MO_CKDN230005698-1A_H3TLGDSX7_L1_trimgalore/LO_MO_CKDN230005698-1A_H3TLGDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,7372666
6,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO/LO_MO_CKDN230005698-1A_H5HVHDSX7_L1_trimgalore/LO_MO_CKDN230005698-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO/LO_MO_CKDN230005698-1A_H5HVHDSX7_L1_trimgalore/LO_MO_CKDN230005698-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,5105101
7,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO/LO_MO_CKDN230005698-1A_H5M5FDSX7_L2_trimgalore/LO_MO_CKDN230005698-1A_H5M5FDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_MO/LO_MO_CKDN230005698-1A_H5M5FDSX7_L2_trimgalore/LO_MO_CKDN230005698-1A_H5M5FDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,123

In [5]:
sort!(pilot_sample_table, "filesize")

Row,sample_ID,forward_reads,reverse_reads,filesize
,String,String,String,Int64
1,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,104396
2,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1_trimgalore/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1_trimgalore/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,367580
3,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2_trimgalore/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2_trimgalore/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,370684
4,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HYWDSX7_L2_trimgalore/RI_UC_CKDN230005693-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HYWDSX7_L2_trimgalore/RI_UC_CKDN230005693-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,373537
5,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2_trimgalore/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/LO_FA/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2_trimgalore/LO_FA_CKDN230005699-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,396783
6,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/VA_PROB,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/VA_PROB/VA_PROB_CKDN230005701-1A_H5HYWDSX7_L2_trimgalore/VA_PROB_CKDN230005701-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/VA_PROB/VA_PROB_CKDN230005701-1A_H5HYWDSX7_L2_trimgalore/VA_PROB_CKDN230005701-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,468990
7,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_MAun,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_MAun/RI_MAun_CKDN230005689-1A_H5HYWDSX7_L2_trimgalore/RI_MAun_CKDN230005689-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_MAun/RI_MAun_CKDN230005689-1A_H5HYWDSX7_L2_trimgalore/RI_MAun_CKDN230005689-1A_H5HYWDSX7_L2.GCF_000001405.40_GRC

In [6]:
kraken_directories = replace.(pilot_sample_table[!, "forward_reads"], ".1.fq.gz" => "_kraken")
pilot_sample_table[!, "kraken_results"] = [first(filter(x -> occursin(r"k2_pluspfp_20231009\.kraken\-report\.tsv$", x), readdir(kraken_directory, join=true))) for kraken_directory in kraken_directories]
pilot_sample_table[!, "mapped_reads"] = replace.(pilot_sample_table[!, "forward_reads"], ".1.fq.gz" => ".nt.fna.gz.xsr.I51G.mmi.minimap2.sam.gz")
pilot_sample_table

Row,sample_ID,forward_reads,reverse_reads,filesize,kraken_results,mapped_reads
,String,String,String,Int64,String,String
1,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,104396,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped_kraken/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.sam.gz
2,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1_trimgalore/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1_trimgalore/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,367580,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1_trimgalore/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped_kraken/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1_trimgalore/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.sam.gz
3,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2_trimgalore/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.1.fq.gz,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2_trimgalore/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.2.fq.gz,370684,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2_trimgalore/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped_kraken/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.k2_pluspfp_20231009.kraken-report.tsv,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2_trimgalore/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.sam.gz
4,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC,/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HYWDSX7_L2_trimgalore/RI_UC_CKDN230005693-1A_H5HYWDSX7

In [12]:
kraken_reports = Dict{String, DataFrames.DataFrame}()
ProgressMeter.@showprogress for row in DataFrames.eachrow(pilot_sample_table)
    kraken_report_file = row["kraken_results"]
    kraken_reports[kraken_report_file] = Mycelia.read_kraken_report(kraken_report_file)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


In [13]:
kraken_reports

Dict{String, DataFrames.DataFrame} with 51 entries:
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame…
  "/oak/stanford/scg/lab_m… => 47442×6 DataFrame

In [27]:
# import Printf

# function parse_xam_to_alignment_metadata(input_file::String)
#     Mycelia.add_bioconda_env("samtools")
#     # Mycelia.add_bioconda_env("pigz")
#     # Determine the file extension
#     file_ext = splitext(input_file)[2]

#     # Step 1: Remove header and convert to SAM format
#     if file_ext == ".bam"
#         p1 = `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header $input_file`
#     elseif file_ext == ".sam"
#         p1 = `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header $input_file`
#     elseif file_ext == ".gz" && endswith(input_file, ".sam.gz")
#         # p1 = pipeline(`$(Mycelia.CONDA_RUNNER) run --live-stream -n pigz pigz -dc $input_file`, `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header`)
#         p1 = pipeline(`gzip -dc $input_file`, `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header`)
#     else
#         Printf.@printf("Unsupported file format: %s\n", file_ext)
#         return
#     end
    
#     record_table = DataFrames.DataFrame(
#         template = String[],
#         flag = UInt16[],
#         reference = Union{Missing, String}[],
#         position = Union{Missing, UnitRange{Int}}[],
#         mappingquality = Union{Missing, UInt8}[],
#         ismapped = Union{Missing, Bool}[],
#         isprimary = Union{Missing, Bool}[],
#         alignment_score = Union{Missing, Int}[],
#     )
#     reader = XAM.SAM.Reader(open(p1))
#     for record in reader
#         if XAM.SAM.ismapped(record)
#             row = (
#                 template = XAM.SAM.tempname(record),
#                 flag = XAM.flag(record),
#                 reference = XAM.SAM.refname(record),
#                 position = XAM.SAM.position(record):XAM.SAM.rightposition(record),
#                 mappingquality = XAM.SAM.mappingquality(record),
#                 ismapped = XAM.SAM.ismapped(record),
#                 isprimary = XAM.SAM.isprimary(record),
#                 alignment_score = record["AS"],
#             )
#             push!(record_table, row, promote=true)
#         else
#             row = (
#                 template = XAM.SAM.tempname(record),
#                 flag = XAM.flag(record),
#                 reference = missing,
#                 position = missing,
#                 mappingquality = missing,
#                 ismapped = missing,
#                 isprimary = missing,
#                 alignment_score = missing,
#             )
#             push!(record_table, row, promote=true)
#         end
#     end
#     close(reader)
#     return record_table
# end

# @time parse_xam(pilot_sample_table[1, "mapped_reads"])

182.676047 seconds (41.45 k allocations: 3.208 MiB)


Row,template,flag,reference,position,mappingquality,ismapped,isprimary,alignment_score
,String,UInt16,String?,UnitRang…?,UInt8?,Bool?,Bool?,Int64?
1,A00738:618:H5HYWDSX7:2:1101:21070:16250,115,MH534845.1,13124:13272,0,true,true,298
2,A00738:618:H5HYWDSX7:2:1101:21070:16250,179,BX546470.1,4966:5115,0,true,true,300
3,A00738:618:H5HYWDSX7:2:1102:18584:12038,89,OU452229.1,3078990:3079014,0,true,true,50
4,A00738:618:H5HYWDSX7:2:1102:18584:12038,165,missing,missing,missing,missing,missing,missing
5,A00738:618:H5HYWDSX7:2:1102:6614:26412,99,BX546470.1,7429:7578,0,true,true,300
6,A00738:618:H5HYWDSX7:2:1102:6614:26412,147,BX546470.1,7644:7792,0,true,true,298
7,A00738:618:H5HYWDSX7:2:1102:6406:27461,115,MH534845.1,10434:10583,0,true,true,300
8,A00738:618:H5HYWDSX7:2:1102:6406:27461,179,BX546470.1,7644:7792,0,true,true,298
9,A00738:618:H5HYWDSX7:2:1105:10773:25441,115,CP068256.2,49481909:49482058,0,true,true,300


In [30]:
import Printf

function parse_xam_to_alignment_metadata(input_file::String; output_file::String = "")
    # Determine the output filename
    if isempty(output_file)
        base_name = basename(input_file)
        # Remove the extension
        if endswith(base_name, ".sam") || endswith(base_name, ".bam")
            file_no_ext = base_name[1:end-4]
        elseif endswith(base_name, ".sam.gz")
            file_no_ext = base_name[1:end-7]
        else
            Printf.@printf("Unsupported file format for file: %s\n", input_file)
            return
        end
        output_file = joinpath(dirname(input_file), file_no_ext * ".mapping-summary-data.arrow")
    end

    # Check if the output file already exists
    if isfile(output_file)
        Printf.@printf("Output file %s already exists. Skipping processing.\n", output_file)
        return output_file
    end

    Mycelia.add_bioconda_env("samtools")

    # Determine the file extension and prepare the command
    if endswith(input_file, ".bam") || endswith(input_file, ".sam")
        p1 = `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header $input_file`
    elseif endswith(input_file, ".sam.gz")
        p1 = pipeline(`gzip -dc $input_file`, `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header`)
    else
        Printf.@printf("Unsupported file format: %s\n", input_file)
        error()
    end

    # Initialize arrays to collect data efficiently
    templates = String[]
    flags = UInt16[]
    references = Vector{Union{Missing, String}}()
    position_starts = Vector{Union{Missing, Int}}()
    position_ends = Vector{Union{Missing, Int}}()
    mappingqualities = Vector{Union{Missing, UInt8}}()
    ismapped_array = Bool[]
    isprimary_array = Vector{Union{Missing, Bool}}()
    alignment_scores = Vector{Union{Missing, Int}}()

    reader = XAM.SAM.Reader(open(p1))
    for record in reader
        push!(templates, XAM.SAM.tempname(record))
        push!(flags, XAM.flag(record))
        is_mapped = XAM.SAM.ismapped(record)
        push!(ismapped_array, is_mapped)
        if is_mapped
            push!(references, XAM.SAM.refname(record))
            push!(position_starts, XAM.SAM.position(record))
            push!(position_ends, XAM.SAM.rightposition(record))
            push!(mappingqualities, XAM.SAM.mappingquality(record))
            push!(isprimary_array, XAM.SAM.isprimary(record))
            push!(alignment_scores, record["AS"])
        else
            push!(references, missing)
            push!(position_starts, missing)
            push!(position_ends, missing)
            push!(mappingqualities, missing)
            push!(isprimary_array, missing)
            push!(alignment_scores, missing)
        end
    end
    close(reader)

    # Create DataFrame from the collected data
    record_table = DataFrames.DataFrame(
        template = templates,
        flag = flags,
        reference = references,
        position_start = position_starts,
        position_end = position_ends,
        mappingquality = mappingqualities,
        ismapped = ismapped_array,
        isprimary = isprimary_array,
        alignment_score = alignment_scores
    )

    # Write the DataFrame to an Arrow file
    Arrow.write(output_file, record_table)
    Printf.@printf("Data successfully written to %s\n", output_file)
    return output_file
end

# Example usage:
@time parse_xam_to_alignment_metadata(pilot_sample_table[1, "mapped_reads"])

Data successfully written to /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.mapping-summary-data.arrow
198.478915 seconds (11.81 M allocations: 801.188 MiB, 0.47% gc time, 1.32% compilation time)


ERROR conda.cli.main_run:execute(124): `conda run samtools view --no-header` failed. (See above for error)


"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.mapping-summary-data.arrow"

In [33]:
# table = DataFrames.DataFrame(Arrow.Table("/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.mapping-summary-data.arrow"))

In [ ]:
# import Printf

# function parse_xam_to_alignment_metadata(input_file::String)
#     Mycelia.add_bioconda_env("samtools")
#     # Determine the file extension
#     file_ext = splitext(input_file)[2]

#     # Step 1: Remove header and convert to SAM format
#     if file_ext == ".bam"
#         p1 = `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header $input_file`
#     elseif file_ext == ".sam"
#         p1 = `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header $input_file`
#     elseif file_ext == ".gz" && endswith(input_file, ".sam.gz")
#         p1 = pipeline(`$(Mycelia.CONDA_RUNNER) run --live-stream -n pigz pigz -dc $input_file`, `$(Mycelia.CONDA_RUNNER) run --live-stream -n samtools samtools view --no-header`)
#     else
#         Printf.@printf("Unsupported file format: %s\n", file_ext)
#         return
#     end
    
#     record_table = DataFrames.DataFrame(
#         template = String[],
#         flag = UInt16[],
#         reference = Union{Missing, String}[],
#         position = Union{Missing, UnitRange{Int}}[],
#         mappingquality = Union{Missing, UInt8}[],
#         ismapped = Union{Missing, Bool}[],
#         isprimary = Union{Missing, Bool}[],
#         alignment_score = Union{Missing, Int}[],
#     )
#     reader = XAM.SAM.Reader(open(p1))
#     for record in reader
#         if XAM.SAM.ismapped(record)
#             row = (
#                 template = XAM.SAM.tempname(record),
#                 flag = XAM.flag(record),
#                 reference = XAM.SAM.refname(record),
#                 position = XAM.SAM.position(record):XAM.SAM.rightposition(record),
#                 mappingquality = XAM.SAM.mappingquality(record),
#                 ismapped = XAM.SAM.ismapped(record),
#                 isprimary = XAM.SAM.isprimary(record),
#                 alignment_score = record["AS"],
#             )
#             push!(record_table, row, promote=true)
#         else
#             row = (
#                 template = XAM.SAM.tempname(record),
#                 flag = XAM.flag(record),
#                 reference = missing,
#                 position = missing,
#                 mappingquality = missing,
#                 ismapped = missing,
#                 isprimary = missing,
#                 alignment_score = missing,
#             )
#             push!(record_table, row, promote=true)
#         end
#     end
#     close(reader)
#     return record_table
# end

# @time parse_xam_to_alignment_metadata(pilot_sample_table[1, "mapped_reads"])

In [34]:
ProgressMeter.@showprogress for row in DataFrames.eachrow(pilot_sample_table)
    @time parse_xam_to_alignment_metadata(row["mapped_reads"])
end

Progress:  53%|█████████████████████▊                   |  ETA: 1:11:53

Output file /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_3BRO/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2_trimgalore/RI_3BRO_CKDN230005692-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.mapping-summary-data.arrow already exists. Skipping processing.
  0.000287 seconds (31 allocations: 2.844 KiB)
Data successfully written to /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1_trimgalore/RI_UC_CKDN230005693-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.mapping-summary-data.arrow
189.921674 seconds (36.07 k allocations: 7.092 MiB)
Data successfully written to /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_PROB/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2_trimgalore/RI_PROB_CKDN230005690-1A_H5HYWDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted

Progress: 100%|█████████████████████████████████████████| Time: 2:32:59


Data successfully written to /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_1BRO/RI_1BRO_CKDN230005691-1A_H5HVHDSX7_L1_trimgalore/RI_1BRO_CKDN230005691-1A_H5HVHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.mapping-summary-data.arrow
186.010948 seconds (209.47 k allocations: 47.597 MiB)
Data successfully written to /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_UC/RI_UC_CKDN230005693-1A_H5HNJDSX7_L2_trimgalore/RI_UC_CKDN230005693-1A_H5HNJDSX7_L2.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.fna.gz.xsr.I51G.mmi.minimap2.mapping-summary-data.arrow
187.806722 seconds (234.42 k allocations: 54.387 MiB)
Data successfully written to /oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/me-cfs/data/samples/RI_MO/RI_MO_CKDN230005688-1A_H5MWHDSX7_L1_trimgalore/RI_MO_CKDN230005688-1A_H5MWHDSX7_L1.GCF_000001405.40_GRCh38.p14_genomic.fna.sorted.bam.unmapped.nt.

In [ ]:
# xam_reports = Dict{String, DataFrames.DataFrame}()

# # 171.163483 seconds (41.07 k allocations: 3.105 MiB)
# # 171.008516 seconds (134.05 k allocations: 8.693 MiB)
# # 172.706550 seconds (148.56 k allocations: 10.103 MiB)
# # 172.686919 seconds (150.53 k allocations: 9.917 MiB)
# # 178.258223 seconds (175.89 k allocations: 11.591 MiB)
# # 172.368465 seconds (186.06 k allocations: 11.799 MiB)
# # 170.123428 seconds (208.50 k allocations: 13.497 MiB)
# # 170.398414 seconds (299.64 k allocations: 19.384 MiB)
# # 179.777363 seconds (276.74 k allocations: 16.656 MiB)

# ProgressMeter.@showprogress for row in DataFrames.eachrow(pilot_sample_table)
#     xam_file = row["mapped_reads"]
#     @time parsed_report = parse_xam(xam_file)
#     xam_reports[xam_file] = parsed_report
# end

Progress:  18%|███████▎                                 |  ETA: 2:01:13

171.163483 seconds (41.07 k allocations: 3.105 MiB)
171.008516 seconds (134.05 k allocations: 8.693 MiB)
172.706550 seconds (148.56 k allocations: 10.103 MiB)
172.686919 seconds (150.53 k allocations: 9.917 MiB)
178.258223 seconds (175.89 k allocations: 11.591 MiB)
172.368465 seconds (186.06 k allocations: 11.799 MiB)
170.123428 seconds (208.50 k allocations: 13.497 MiB)
170.398414 seconds (299.64 k allocations: 19.384 MiB)
179.777363 seconds (276.74 k allocations: 16.656 MiB)



CondaError: KeyboardInterrupt



In [8]:
# pilot_sample_table[!, "%classified_kraken"] .= 0.0
# pilot_sample_table[!, "%classified_mapping"] .= 0.0
# for (i, row) in enumerate(DataFrames.eachrow(pilot_sample_table))
#     kraken_report = 
#     pilot_sample_table[i, "%classified_kraken"] = kraken_report[findfirst(kraken_report[!, "ncbi_taxonid"] .== 1), "percentage_of_fragments_at_or_below_taxon"]
# end
# pilot_sample_table[!, "%classified_kraken"]

In [9]:
# function parse_xam_to_classification_stats(xam)
#     record_table = DataFrames.DataFrame(
#         template = String[],
#         flag = UInt16[],
#         reference = String[],
#         position = UnitRange{Int}[],
#         mappingquality = UInt8[],
#         tlen = Int[],
#         alignlength = Int[],
#         ismapped = Bool[],
#         isprimary = Bool[],
#         alignment_score = Int[],
#         mismatches = Int[]
#     )
#     if occursin(r"\.bam$", xam)
#         MODULE = XAM.BAM
#         io = open(xam)
#     elseif occursin(r"\.sam$", xam)
#         MODULE = XAM.SAM
#         io = open(xam)
#     elseif occursin(r"\.sam.gz$", xam)
#         MODULE = XAM.SAM
#         io = CodecZlib.GzipDecompressorStream(open(xam))
#     else
#         error("unrecognized file extension in file: $xam")
#     end
#     # filter out header lines
#     reader = MODULE.Reader(IOBuffer(join(Iterators.filter(line -> !startswith(line, '@'), eachline(io)), '\n')))
#     # reader = MODULE.Reader(io)
#     mapped_ids = Set{String}()
#     unmapped_ids = Set{String}()
#     for record in reader
#         if XAM.SAM.ismapped(record)
#             push!(mapped_ids, XAM.SAM.tempname(record))
#         else
#             push!(unmapped_ids, XAM.SAM.tempname(record))
#         end
#     end
#     close(io)
#     filter!(read -> !(read in mapped_ids), unmapped_ids)
#     return (;mapped_ids, unmapped_ids)
# end

# mapping_id_sets = parse_xam_to_classification_stats(pilot_sample_table[1, "mapped_reads"])

In [ ]:
ProgressMeter.@showprogress for (i, row) in enumerate(DataFrames.eachrow(pilot_sample_table))
    sam_file = row["mapped_reads"]
    mapping_id_sets = parse_xam_to_classification_stats(sam_file)
    percent_mapped = length(mapping_id_sets.mapped_ids) / (length(mapping_id_sets.mapped_ids) + length(mapping_id_sets.unmapped_ids)) * 100
    pilot_sample_table[i, "%classified_mapping"] = percent_mapped
end

pilot_sample_table[!, "%classified_mapping"]

In [ ]:
p = StatsPlots.scatter(
    pilot_sample_table[!, "%classified_kraken"],
    pilot_sample_table[!, "%classified_mapping"],
    xlabel = "% classified kraken NT (OLD)",
    ylabel = "% classified mapping NT (NEW)",
    title = "relative classification gains for NONHUMAN reads",
    xlims=[0, 100],
    ylims=[0, 100])
StatsPlots.plot(p, 0:100, 0:100)